In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
#from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import make_scorer, accuracy_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import initializers

from matplotlib import pyplot
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
#LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

# One-h-k

In [2]:
def one_hot_encode(dataset, variables):
    for variable in variables:
        dummies = pd.get_dummies(dataset[variable], prefix=variable).astype(int)
        dataset = pd.concat([dataset, dummies], axis=1)
        dataset.drop(columns=[variable], inplace=True)
    return dataset

# Search Function

In [ ]:
def search(x_tr, y_tr, x_vl, y_vl, lr, functions1, functions2, batch_size, units, epochs):
    # Record the best score and corresponding hyperparameters
    best_score = None
    best_params = {}
    best_model_1 = None
    best_accuracy = None

    # Manual grid search
    for lr_temp in lr:
        for n_unit in units:
            for function_temp1 in functions1:
                for function_temp2 in functions2:
                    for bs_temp in batch_size:
                        # Build the model
                        model = Sequential()
                        model.add(Dense(n_unit, input_shape=(x_tr.shape[1],), activation=function_temp1, kernel_initializer=initializers.glorot_uniform(seed=0)))  # primo livello che ha il numero di neuroni (4) uguale al numero di input features. poi viene specificato il numero delle colonne input
                        model.add(Dense(1, activation=function_temp2))  # secondo layer un solo neurone per task di classificazione

                        opt = tf.keras.optimizers.Adam(lr_temp)
                        model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
                        history = model.fit(
                            x_tr, y_tr,
                            epochs=epochs,
                            batch_size=bs_temp,
                            verbose=1
                        )

                        # Evaluate the model
                        loss_tr, accuracy_tr = model.evaluate(x_tr, y_tr, verbose=0)
                        loss_vl, accuracy_vl = model.evaluate(x_vl, y_vl, verbose=0)

                        # Update best score and parameters if the current model is better
                        if best_accuracy is None or (accuracy_vl > best_accuracy):
                            best_accuracy = accuracy_vl
                            best_params = {
                                'learning_rate': lr_temp,
                                'batch_size': bs_temp,
                                'function1': function_temp1,
                                'function2': function_temp2,
                                'n_units': n_unit
                            }
                            best_accuracy_vl = accuracy_vl
                            best_accuracy_tr = accuracy_tr
                            best_loss_tr = loss_tr
                            best_loss_vl = loss_vl

    print("Best hyperparameters:")
    print(best_params)
    print('Best accuracy  tr', best_accuracy_tr)
    print('Best accuracy  vl', best_accuracy_vl)
    print('Best loss tr', best_loss_tr)
    print('Best loss  vl', best_loss_vl)

    return best_params


# MONK 1

In [12]:
df = pd.read_csv('MONK/monks-1.train', delim_whitespace=True, low_memory=False, header = None, names = ['y',1,2,3,4,5,6], usecols=[0,1,2,3,4,5,6])
df_2 = pd.read_csv('MONK/monks-1.test', delim_whitespace=True, low_memory=False, header = None, names = ['y',1,2,3,4,5,6], usecols=[0,1,2,3,4,5,6])
#124 righe df
#432 righe df_2

## Applicazione one hot encoding

In [15]:
monk1_tr=one_hot_encode(df, [1,2,3,4,5,6])
monk1_ts=one_hot_encode(df_2, [1,2,3,4,5,6])

## Split dataset

In [17]:
training_1_x, validation_1_x, training_1_y, validation_1_y = train_test_split(monk1_tr, df[['y']], stratify =None, test_size=0.30, random_state = 42, shuffle=False)

In [ ]:
x_1_ts = monk1_ts.iloc[:,1:].values #ottiene tutte le colonne con indice >=1, scartando la target label
y_1_ts = monk1_ts.iloc[:,0].values # ottiene solo colonne con indice 0, solo la target label
x_1_tr = training_1_x.iloc[:,1:].values
y_1_tr = training_1_y.iloc[:,:].values
x_1_vl = validation_1_x.iloc[:,1:].values
y_1_vl = validation_1_y.iloc[:,:].values

x_1_ds = monk1_tr.iloc[:,1:].values
y_1_ds = monk1_tr.iloc[:,0].values

##  Search

In [ ]:
%%time

# Define hyperparameters to search
lr = [0.002, 0.006, 0.008]
functions1 = ['tanh', 'sigmoid']
functions2 = ['tanh', 'sigmoid','softmax']

batch_size = [4,8,12]
units = [4]

best_params_monk1 = search(x_tr=x_1_tr ,y_tr=y_1_tr, x_vl=x_1_vl,y_vl=y_1_vl, lr=lr, functions1= functions1,functions2=functions2, batch_size= batch_size, units= units, epochs= 80)

In [ ]:
best_model = Sequential()
best_model.add(Dense(best_params_monk1['n_units'], input_shape=(x_1_tr.shape[1],), activation=best_params_monk1['function1'],kernel_initializer=initializers.glorot_uniform(seed=0))) # primo livello che ha il numero di neuroni (4) uguale al numero di                                                                          # input features. poi viene specificato il numero delle colonne input
best_model.add(Dense(1, activation= best_params_monk1['function2'])) # secondo layer un solo neurone per task di classificazione

opt = tf.keras.optimizers.Adam(best_params_monk1['learning_rate'])
best_model.compile(loss='mean_squared_error', optimizer = opt, metrics=['accuracy'])
history_best = best_model.fit(
              x_1_ds, y_1_ds, validation_data=(x_1_ts,y_1_ts),
              epochs=80,
              batch_size= best_params_monk1['batch_size'], verbose=2
          )

training_mse,training_accuracy = best_model.evaluate(x_1_ds, y_1_ds, verbose=0)
test_mse,test_accuracy = best_model.evaluate(x_1_ts, y_1_ts,verbose=0)
print('Training MSE: %.6f Test MSE: %.6f'%(training_mse,test_mse))
print('Training accuracy: %.4f Test accuracy: %.4f'%(training_accuracy,test_accuracy))



## Risultati

In [ ]:
print('Training MSE: %.6f Test MSE: %.6f'%(training_mse,test_mse))
print('Training accuracy: %.4f Test accuracy: %.4f'%(training_accuracy,test_accuracy))
pyplot.title('MONK 1')
pyplot.plot(history_best.history['loss'], label='training')
pyplot.plot(history_best.history['val_loss'], label='test',linestyle="dashed")
pyplot.xlabel('epoch')
pyplot.ylabel('MSE')
pyplot.legend()
pyplot.show()

In [ ]:
pyplot.title('MONK 1 ACCURACY')
pyplot.plot(history_best.history['accuracy'], label='train')
pyplot.plot(history_best.history['val_accuracy'], label='test',linestyle="dashed")
pyplot.xlabel('epoch')
pyplot.ylabel('ACCURACY')
pyplot.legend()
pyplot.show()

In [ ]:
import matplotlib.pyplot as plt
best_model.fit(x_1_tr, y_1_tr)
predictions = best_model.predict(x_1_ts)
# Plot for the first model
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(y_1_ts, predictions)
plt.title("Model GridSearch - Predicted vs Actual")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")


plt.show()


# MONK 2

In [ ]:
df = pd.read_csv('MONK/monks-2.train', delim_whitespace=True, low_memory=False, header = None, names = ['y',1,2,3,4,5,6], usecols=[0,1,2,3,4,5,6])
df_2 = pd.read_csv('MONK/monks-2.test', delim_whitespace=True, low_memory=False, header = None, names = ['y',1,2,3,4,5,6], usecols=[0,1,2,3,4,5,6])


## Applicazione one hot encoding e split dataset

In [ ]:
monk2_tr=one_hot_encode(df, [1,2,3,4,5,6])
monk2_ts=one_hot_encode(df_2, [1,2,3,4,5,6])
training_2_x, validation_2_x, training_2_y, validation_2_y = train_test_split(monk2_tr, df[['y']], stratify =None, test_size=0.30, random_state = 42, shuffle=False)

x_2_ts = monk2_ts.iloc[:,1:].values #ottiene tutte le colonne con indice >=1, scartando la target label
y_2_ts = monk2_ts.iloc[:,0].values # ottiene solo colonne con indice 0, solo la target label

x_2_tr = training_2_x.iloc[:,1:].values
y_2_tr = training_2_y.iloc[:,:].values
x_2_vl = validation_2_x.iloc[:,1:].values
y_2_vl = validation_2_y.iloc[:,:].values

x_2_ds = monk2_tr.iloc[:,1:].values
y_2_ds = monk2_tr.iloc[:,0].values


##  Search

In [ ]:
%%time
# Define hyperparameters to search
lr = [0.002, 0.008, 0.01]
functions1 = ['tanh', 'sigmoid']
functions2 = ['tanh', 'sigmoid','softmax']
batch_size = [4,8,12]
units = [4]

best_params_monk2 = search(x_tr=x_2_tr ,y_tr=y_2_tr, x_vl=x_2_vl,y_vl=y_2_vl, lr=lr,functions1= functions1,functions2=functions2, batch_size= batch_size, units= units, epochs= 40)


In [ ]:
best_model = Sequential()
best_model.add(Dense(best_params_monk2['n_units'], input_shape=(x_2_tr.shape[1],), activation=best_params_monk2['function1'],kernel_initializer=initializers.glorot_uniform(seed=0))) # primo livello che ha il numero di neuroni (4) uguale al numero di                                                                          # input features. poi viene specificato il numero delle colonne input
best_model.add(Dense(1, activation= best_params_monk2['function2'])) # secondo layer un solo neurone per task di classificazione

opt = tf.keras.optimizers.Adam(best_params_monk2['learning_rate'])
best_model.compile(loss='mean_squared_error', optimizer = opt, metrics=['accuracy'])
history_best_2 = best_model.fit(
              x_2_ds, y_2_ds, validation_data=(x_2_ts,y_2_ts),
              epochs=40,
              batch_size= best_params_monk2['batch_size'], verbose=2
          )

training_mse_2,training_accuracy_2 = best_model.evaluate(x_2_ds, y_2_ds, verbose=0)
test_mse_2,test_accuracy_2 = best_model.evaluate(x_2_ts, y_2_ts,verbose=0)
print('Training MSE: %.6f Test MSE: %.6f'%(training_mse_2,test_mse_2))
print('Training accuracy: %.4f Test accuracy: %.4f'%(training_accuracy_2,test_accuracy_2))



## Risultati

In [ ]:

print('Training MSE: %.6f Test MSE: %.6f'%(training_mse_2,test_mse_2))
print('Training accuracy: %.4f Test accuracy: %.4f'%(training_accuracy_2,test_accuracy_2))
pyplot.title('MONK 2')
pyplot.plot(history_best_2.history['loss'], label='training')
pyplot.plot(history_best_2.history['val_loss'], label='test',linestyle="dashed")
pyplot.xlabel('epoch')
pyplot.ylabel('MSE')
pyplot.legend()
pyplot.show()


In [ ]:
#Evaluate model
print('Training MSE: %.6f Test MSE: %.6f'%(training_mse_2,test_mse_2))
print('Training accuracy: %.4f Test accuracy: %.4f'%(training_accuracy_2,test_accuracy_2))
pyplot.title('MONK 2 ACCURACY')
pyplot.plot(history_best_2.history['accuracy'], label='training')
pyplot.plot(history_best_2.history['val_accuracy'], label='test',linestyle="dashed")
pyplot.xlabel('epoch')
pyplot.ylabel('MSE')
pyplot.legend()
pyplot.show()


# MONK 3

In [ ]:
df = pd.read_csv('MONK/monks-3.train', delim_whitespace=True, low_memory=False, header = None, names = ['y',1,2,3,4,5,6], usecols=[0,1,2,3,4,5,6])
df_2 = pd.read_csv('MONK/monks-3.test', delim_whitespace=True, low_memory=False, header = None, names = ['y',1,2,3,4,5,6], usecols=[0,1,2,3,4,5,6])


## Applicazione one hot encoding e split dataset

In [ ]:
monk3_tr=one_hot_encode(df, [1,2,3,4,5,6])
monk3_ts=one_hot_encode(df_2, [1,2,3,4,5,6])
training_3_x, validation_3_x, training_3_y, validation_3_y = train_test_split(monk3_tr, df[['y']], stratify =None, test_size=0.30, random_state = 42, shuffle=False)

x_3_ts = monk3_ts.iloc[:,1:].values #ottiene tutte le colonne con indice >=1, scartando la target label
y_3_ts = monk3_ts.iloc[:,0].values # ottiene solo colonne con indice 0, solo la target label


x_3_tr = training_3_x.iloc[:,1:].values
y_3_tr = training_3_y.iloc[:,:].values
x_3_vl = validation_3_x.iloc[:,1:].values
y_3_vl = validation_3_y.iloc[:,:].values

x_3_ds = monk3_tr.iloc[:,1:].values
y_3_ds = monk3_tr.iloc[:,0].values



##  Search

In [ ]:
%%time

# Define hyperparameters to search
lr = [0.02, 0.005, 0.01]
momentum = [0] # use higher on start of net (usually done!)
functions1 = ['tanh', 'sigmoid']
functions2 = ['tanh', 'sigmoid','softmax']
batch_size = [4,8,12,16]
units = [4,8]

best_params_monk3 = search(x_tr=x_3_tr ,y_tr=y_3_tr, x_vl=x_3_vl,y_vl=y_3_vl, lr=lr, functions1= functions1,functions2=functions2, batch_size= batch_size, units= units, epochs= 50)


In [ ]:
best_model = Sequential()
best_model.add(Dense(best_params_monk3['n_units'], input_shape=(x_3_tr.shape[1],), activation=best_params_monk3['function1'],kernel_initializer=initializers.glorot_uniform(seed=0))) # primo livello che ha il numero di neuroni (4) uguale al numero di                                                                          # input features. poi viene specificato il numero delle colonne input
best_model.add(Dense(1, activation= best_params_monk3['function2'])) # secondo layer un solo neurone per task di classificazione

opt = tf.keras.optimizers.Adam(best_params_monk3['learning_rate'])
best_model.compile(loss='mean_squared_error', optimizer = opt, metrics=['accuracy'])
history_best_3 = best_model.fit(
              x_3_ds, y_3_ds, validation_data=(x_3_ts,y_3_ts),
              epochs=50,
              batch_size= best_params_monk3['batch_size'], verbose=2
          )

training_mse_3,training_accuracy_3 = best_model.evaluate(x_3_ds, y_3_ds, verbose=0)
test_mse_3,test_accuracy_3 = best_model.evaluate(x_3_ts, y_3_ts,verbose=0)
print('Training MSE: %.6f Test MSE: %.6f'%(training_mse_3,test_mse_3))
print('Training accuracy: %.4f Test accuracy: %.4f'%(training_accuracy_3,test_accuracy_3))



## Risultati

In [ ]:

print('Training MSE: %.4f Test MSE: %.4f'%(training_mse_3,test_mse_3))
print('Training accuracy: %.4f Test accuracy: %.4f'%(training_accuracy_3,test_accuracy_3))
pyplot.title('MONK 3')
pyplot.plot(history_best_3.history['loss'], label='training')
pyplot.plot(history_best_3.history['val_loss'], label='test', linestyle="dashed")
pyplot.xlabel('epoch')
pyplot.ylabel('MSE')
pyplot.legend()
pyplot.show()

In [ ]:

print('Training MSE: %.4f Test MSE: %.4f'%(training_mse_3,test_mse_3))
print('Training accuracy: %.4f Test accuracy: %.4f'%(training_accuracy_3,test_accuracy_3))
pyplot.title('MONK 3 ACCURACY')
pyplot.plot(history_best_3.history['accuracy'], label='training')
pyplot.plot(history_best_3.history['val_accuracy'], label='test', linestyle="dashed")
pyplot.xlabel('epoch')
pyplot.ylabel('MSE')
pyplot.legend()
pyplot.show()